**Install the transformers and datasets for the language modelling tasks**

In [1]:
! pip install datasets transformers

     |████████████████████████████████| 269 kB 5.2 MB/s 
     |████████████████████████████████| 2.8 MB 41.4 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 119 kB 39.0 MB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 51.6 MB/s 
     |████████████████████████████████| 1.3 MB 37.3 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 636 kB 43.8 MB/s 
     |████████████████████████████████| 3.3 MB 35.1 MB/s 
     |████████████████████████████████| 142 kB 48.2 MB/s 
     |████████████████████████████████| 294 kB 49.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**Singing in the HuggingFace API login for the authentication**

In [2]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: VirenS13117
Password: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


**Setting up Git LFS**

In [3]:
!apt install git-lfs
!git config --global user.email "virender13117@gmail.com"
!git config --global user.name "VirenS13117"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 3s (657 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


**Checking up the Transformers version**

In [4]:
import transformers

In [5]:
transformers.__version__

'4.10.2'

**Now we fine tune a Languange Model**

**Task 1 : Fine tuning a masked language modeling task** \\
A Masked Language Modeling task has a masked tokens and we have access to other tokens before and after the masked token with which we predict the masked token.



Next we prepare the dataset "WikiText2" for this task using the dataset library.

---



In [6]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
#OR
#Using custom dataset
#datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
datasets['train'][3]

{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'}

Following function shows some randomly picked examples in the dataset

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,text
0,"M @-@ 114 was the designation of a former state trunkline highway and planned beltline in the US state of Michigan around the city of Grand Rapids . It was designated by the end of 1929 on various streets in adjoining cities and townships . By the 1940s , sections of it on the west and south sides of Grand Rapids were given new designations and the segment along the east side of town was finished . By late 1945 the highway designation was completely decommissioned in favor of other numbers . M @-@ 114 split into two branches , one running east – west and the other running north – south . The east – west spur routing is now local streets while the rest is part of state highways . \n"
1,"Mole crickets are members of the insect family Gryllotalpidae , in the order Orthoptera ( grasshoppers , locusts and crickets ) . Mole crickets are cylindrical @-@ bodied insects about 3 – 5 centimetres ( 1 @.@ 2 – 2 @.@ 0 in ) long , with small eyes and shovel @-@ like forelimbs highly developed for burrowing . They are present in many parts of the world and where they have been introduced into new regions , may become agricultural pests . \n"
2,= = = Story = = = \n
3,<formula> \n
4,"Abandonments , Bat \n"
5,= = = 2nd Brigade = = = \n
6,
7,= Transportation in Omaha = \n
8,
9,"As of 2007 , Proctor has two separate businesses : a general store ( which also sells gasoline ) and a bed and breakfast . The church which used to host the annual "" Proctor Homecoming "" reunions still stands , but is closed . Barbours has no store or gas station , but does have one church . Barbours is home to the Plunketts Creek Township Volunteer Fire Company and township municipal building ( which houses a small branch library ) . Since 1967 , Barbours has been home to Pneu @-@ Dart , which makes tranquilizer darts and guns for livestock and wildlife capture and control . In 1997 , Pneu @-@ Dart had eight employees . Today much of Plunketts Creek 's watershed is wooded and protected as part of Loyalsock State Forest or Pennsylvania State Game Lands No. 134 . Pennsylvania 's state forests and game lands are managed , and small @-@ scale lumbering operations continue in the watershed today . Barbours has one sawmill , in 1997 it had thirty contract loggers and fifteen employees , with $ 1 @.@ 2 million in annual gross sales . \n"


A preprocessing function that groups the text

In [10]:
def group_texts(examples, block_size=128):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

For masked language modeling (MLM) we are going to preprocessing our dataset with one additional step: we will randomly mask some tokens (by replacing them by [MASK]) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We are going to use the [DistilRoBERTa](https://huggingface.co/distilroberta-base) base model. This is a case-sensitive model, it has has 6 layers, 768 dimension and 12 heads, totalizing 82M parameters.

In [11]:
model_checkpoint = "distilroberta-base"

Now we apply tokenization function, we just need to update our tokenizer to use the checkpoint we just picked:

In [12]:
from transformers import AutoTokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"])


In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


We can now group texts together and chunk them in samples of length block_size. 

In [14]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Dataset is preprocessed now ans we can check our datasets have changed: now the samples contain chunks of block_size contiguous tokens, potentially spanning over several of our original texts.

In [15]:
lm_datasets

DatasetDict({
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 2277
    })
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 19242
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 1988
    })
})

In [16]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' the story runs parallel to the first game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n</s><s> The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with'

Now, we set a new model which will take away from the base checkpoint of the DistillRoberta and then train it on out set of tokens

In [17]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


Now we can add our training arguments for our model. You can also later push your model to the hub with the keywords push_to_hub and assign it an id with push_to_hub_model_id

In [18]:
from transformers import Trainer, TrainingArguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-wikitext2",
)

Assigning these attributes to the Trainer class.

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

Cloning https://huggingface.co/VirenS13117/distilroberta-base-finetuned-wikitext2 into local empty directory.


Finally training our model

In [20]:
trainer.train()

***** Running training *****
  Num examples = 19242
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7218


Epoch,Training Loss,Validation Loss
1,0.004200,0.000082


Saving model checkpoint to test-clm/checkpoint-500
Configuration saved in test-clm/checkpoint-500/config.json
Model weights saved in test-clm/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1000
Configuration saved in test-clm/checkpoint-1000/config.json
Model weights saved in test-clm/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1500
Configuration saved in test-clm/checkpoint-1500/config.json
Model weights saved in test-clm/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-2000
Configuration saved in test-clm/checkpoint-2000/config.json
Model weights saved in test-clm/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1988
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-2500
Configuration saved in test-clm/checkpoint-2500/config.json
Model weights saved in test-clm/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpo

Epoch,Training Loss,Validation Loss
1,0.004200,0.000082
2,0.001300,0.000044
3,0.000800,0.000017


Saving model checkpoint to test-clm/checkpoint-4000
Configuration saved in test-clm/checkpoint-4000/config.json
Model weights saved in test-clm/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-4500
Configuration saved in test-clm/checkpoint-4500/config.json
Model weights saved in test-clm/checkpoint-4500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1988
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-5000
Configuration saved in test-clm/checkpoint-5000/config.json
Model weights saved in test-clm/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-5500
Configuration saved in test-clm/checkpoint-5500/config.json
Model weights saved in test-clm/checkpoint-5500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-6000
Configuration saved in test-clm/checkpoint-6000/config.json
Model weights saved in test-clm/checkpoint-6000/pytorch_model.bin
Saving model checkpoint to test-clm/chec

TrainOutput(global_step=7218, training_loss=0.07024849713030334, metrics={'train_runtime': 2862.3787, 'train_samples_per_second': 20.167, 'train_steps_per_second': 2.522, 'total_flos': 1913931601832448.0, 'train_loss': 0.07024849713030334, 'epoch': 3.0})